In [1]:
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
nltk.download('stopwords')

# Download the stopwords
stopwords = stopwords.words('english')

[nltk_data] Error loading stopwords: <urlopen error [Errno 54]
[nltk_data]     Connection reset by peer>


In [3]:
# from google.colab import files
# uploaded = files.upload()

In [4]:
train_df = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/CSE343-ML-Project/Data/Preprocessed-Data/train.csv')
test_df = pd.read_csv('/Users/arnav/Desktop/MachineLearning/ML_CSE343 Project/CSE343-ML-Project/Data/Preprocessed-Data/test.csv')

In [5]:
train_df.head()

,text
0,omg guys hello father may please acquire nouri...
1,suicidal people sub if need talk messege im
2,existential question the opposite ptsd memory
3,can imagine getting old me neitherwrinkles wei...
4,imagine rich lmao sounds kinda like control wo...


In [6]:
test_df.head()

,text
0,seeing everyone else better real trigger mebas...
1,why cant i ever normalim always pain i dont th...
2,i wasted lifei basically wasted life i never b...
3,the voice tells kill selfthe voice abuser cons...
4,i figured today complete survey get gems downl...


In [7]:
# Print datatypes
print(train_df.dtypes)
print(test_df.dtypes)

# Convert to string
train_df['text'] = train_df['text'].astype(str)
test_df['text'] = test_df['text'].astype(str)

train_df.head()

text    object
dtype: object
text    object
dtype: object


,text
0,omg guys hello father may please acquire nouri...
1,suicidal people sub if need talk messege im
2,existential question the opposite ptsd memory
3,can imagine getting old me neitherwrinkles wei...
4,imagine rich lmao sounds kinda like control wo...


In [8]:
# Preprocessing the data
# Removing the stopwords
train_df['text'] = train_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))
test_df['text'] = test_df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

# Lemmatization
lemmatizer = WordNetLemmatizer()
train_df['text'] = train_df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
test_df['text'] = test_df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Stemming
stemmer = PorterStemmer()
train_df['text'] = train_df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
test_df['text'] = test_df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

# Tokenization
tokenizer = TweetTokenizer()
train_df['text'] = train_df['text'].apply(lambda x: tokenizer.tokenize(x))
test_df['text'] = test_df['text'].apply(lambda x: tokenizer.tokenize(x))

In [9]:
train_df.head()

,text
0,"[omg, guy, hello, father, may, pleas, acquir, ..."
1,"[suicid, peopl, sub, need, talk, messeg, im]"
2,"[existenti, question, opposit, ptsd, memori]"
3,"[imagin, get, old, neitherwrinkl, weight, gain..."
4,"[imagin, rich, lmao, sound, kinda, like, contr..."


In [10]:
# Assuming train_df is your DataFrame containing tokenized text data
sentences = train_df['text'].tolist()

# Train Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=512, window=5, min_count=1, sg=0)  # You can adjust parameters as needed

In [12]:
# Function to average word vectors for a sentence
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,), dtype="float64")
    nwords = 0.
    
    for word in words:
        if word in vocabulary:
            nwords = nwords + 1
            feature_vector = np.add(feature_vector, model.wv[word])
    
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
        
    return feature_vector

# Get vocabulary from Word2Vec model
vocabulary = set(word2vec_model.wv.index_to_key)

# Create word embeddings for train data
train_embeddings = [average_word_vectors(sentence, word2vec_model, vocabulary, 512) for sentence in sentences]

# Assuming test_df is your DataFrame containing tokenized test text data
test_sentences = test_df['text'].tolist()
test_embeddings = [average_word_vectors(sentence, word2vec_model, vocabulary, 512) for sentence in test_sentences]

In [13]:
# Convert to numpy array
train_embeddings = np.array(train_embeddings)
test_embeddings = np.array(test_embeddings)

# Convert to DataFrame
train_embeddings_df = pd.DataFrame(train_embeddings)
test_embeddings_df = pd.DataFrame(test_embeddings)

train_embeddings_df.head()


,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,0.052694,-0.118125,-0.229396,0.291986,0.009241,0.129367,0.071115,-0.209811,-0.082933,0.000301,...,0.083175,0.151262,-0.049164,0.023413,0.065409,0.052793,0.043139,-0.045567,-0.013987,-0.279268
1,-0.012341,-0.075021,-0.352102,0.300257,-0.121494,-0.123827,0.178430,-0.146337,0.180180,0.093710,...,0.131820,0.102414,-0.228749,-0.105752,-0.059067,0.070512,-0.188062,0.126443,-0.125112,-0.117223
2,-0.006011,0.073412,-0.113178,-0.021171,0.031751,0.011544,0.055233,0.095368,-0.020521,0.127266,...,-0.001598,0.052377,-0.040328,0.059277,-0.124763,-0.029379,-0.165305,0.041809,-0.052489,-0.090229
3,0.146414,-0.250966,-0.319383,0.200282,-0.097418,0.239463,0.064298,-0.249808,0.043520,-0.097312,...,0.262556,0.045028,-0.215885,0.129759,-0.042807,0.267330,-0.137384,0.095027,-0.215356,-0.073714
4,0.056085,-0.227654,-0.133726,0.132244,-0.085359,0.212068,0.163954,-0.279782,-0.034780,0.082601,...,0.315412,0.158915,-0.128613,0.042148,0.013348,0.302422,-0.104443,0.037786,-0.201890,-0.073749


In [ ]:
# Save to a CSV file
train_embeddings_df.to_csv("train_word2vec.csv")
test_embeddings_df.to_csv("test_word2vec.csv")